---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train.csv', encoding='latin1')

# Filter out data where violator was found not responsible
train_filtered = train[pd.notnull(train['compliance'])]

train_filtered.set_index(['ticket_id'])

train_filtered.head()

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
5,22046,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","KASIMU, UKWELI",6478.0,NORTHFIELD,NaN,2755.0,E. 17TH,LOG BEACH,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
6,18738,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Deerwood Development Group Inc, Deer",8027.0,BRENTWOOD,NaN,476.0,Garfield,Clinton,...,0.0,855.0,0.0,855.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
7,18735,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Rafee Auto Services L.L.C., RAF",8228.0,MT ELLIOTT,NaN,8228.0,Mt. Elliott,Detroit,...,0.0,140.0,0.0,140.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0


In [3]:
addresses = pd.read_csv('addresses.csv')

addresses.head()

,ticket_id,address
0,22056,"2900 tyler, Detroit MI"
1,27586,"4311 central, Detroit MI"
2,22062,"1449 longfellow, Detroit MI"
3,22084,"1441 longfellow, Detroit MI"
4,22093,"2449 churchill, Detroit MI"


In [4]:
latlons = pd.read_csv('latlons.csv')

# Merge latlons with address
geo = pd.merge(addresses[['ticket_id', 'address']], latlons[['lat', 'lon', 'address']], how='inner', on='address')

geo.head()

,ticket_id,address,lat,lon
0,22056,"2900 tyler, Detroit MI",42.390729,-83.124268
1,77242,"2900 tyler, Detroit MI",42.390729,-83.124268
2,77243,"2900 tyler, Detroit MI",42.390729,-83.124268
3,103945,"2900 tyler, Detroit MI",42.390729,-83.124268
4,138219,"2900 tyler, Detroit MI",42.390729,-83.124268


In [5]:
from sklearn.preprocessing import Imputer

test_raw = pd.read_csv('test.csv')
test_raw = pd.merge(test_raw, geo, how='left', on='ticket_id')
test_raw = test_raw.drop_duplicates('ticket_id', take_last=True)
test_raw = test_raw[['ticket_id', 
                     'fine_amount', 
                     'admin_fee',
                     'state_fee',
                     'late_fee',
                     'discount_amount',
                     'judgment_amount',
                     'lat', 
                     'lon']]

# eliminate NaN error
Imputer = Imputer(missing_values='NaN', strategy='mean', axis=0)
Imputer = Imputer.fit(test_raw)
test_array = Imputer.transform(test_raw)

test = pd.DataFrame(data=test_array,
                    columns=['ticket_id', 
                             'fine_amount', 
                             'admin_fee',
                             'state_fee',
                             'late_fee',
                             'discount_amount',
                             'judgment_amount',
                             'lat', 
                             'lon'])

test['ticket_id'] = test['ticket_id'].astype('int64')

test.head()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:5: FutureWarning: the take_last=True keyword is deprecated, use keep='last' instead


,ticket_id,fine_amount,admin_fee,state_fee,late_fee,discount_amount,judgment_amount,lat,lon
0,284932,200.0,20.0,10.0,20.0,0.0,250.0,42.407581,-82.986642
1,285362,1000.0,20.0,10.0,100.0,0.0,1130.0,42.426239,-83.238259
2,285361,100.0,20.0,10.0,10.0,0.0,140.0,42.426239,-83.238259
3,285338,200.0,20.0,10.0,20.0,0.0,250.0,42.309661,-83.122426
4,285346,100.0,20.0,10.0,10.0,0.0,140.0,42.308830,-83.121116


In [6]:
analysis_vars = pd.merge(train_filtered[['ticket_id', 
                                         'fine_amount', 
                                         'admin_fee',
                                         'state_fee',
                                         'late_fee',
                                         'discount_amount',
                                         'judgment_amount',
                                         'compliance']], geo, how='inner', on='ticket_id')

analysis_vars = analysis_vars.drop_duplicates('ticket_id', take_last=True)
analysis_vars = analysis_vars[['ticket_id', 
                               'fine_amount', 
                               'admin_fee',
                               'state_fee',
                               'late_fee',
                               'discount_amount',
                               'judgment_amount',
                               'lat', 
                               'lon', 
                               'compliance']]

analysis_vars = analysis_vars.dropna(axis=0, how='any')

analysis_vars.head()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:10: FutureWarning: the take_last=True keyword is deprecated, use keep='last' instead


,ticket_id,fine_amount,admin_fee,state_fee,late_fee,discount_amount,judgment_amount,lat,lon,compliance
0,22056,250.0,20.0,10.0,25.0,0.0,305.0,42.390729,-83.124268,0.0
1,27586,750.0,20.0,10.0,75.0,0.0,855.0,42.326937,-83.135118,1.0
2,22046,250.0,20.0,10.0,25.0,0.0,305.0,42.145257,-83.208233,0.0
3,18738,750.0,20.0,10.0,75.0,0.0,855.0,42.433466,-83.023493,0.0
4,18735,100.0,20.0,10.0,10.0,0.0,140.0,42.388641,-83.037858,0.0


In [11]:
def blight_model():
    # This model gives us an AUC of .74
    from sklearn.ensemble import RandomForestClassifier
    
    X_train = analysis_vars.iloc[:,1:-1]
    y_train = analysis_vars.iloc[:,-1]
    X_test = test[['fine_amount', 
                   'admin_fee',
                   'state_fee',
                   'late_fee',
                   'discount_amount',
                   'judgment_amount',
                   'lat', 
                   'lon']] 
    
    random_forest = RandomForestClassifier(n_estimators=100)
    random_forest.fit(X_train, y_train)
    
    # This returns .988 - promising
    acc_random_forest = random_forest.score(X_train, y_train)
    
    y_pred = random_forest.predict(X_test)
    
    probs = random_forest.predict_proba(X_test)
    prob_payment = [p[1] for p in probs]
    
    return pd.Series(prob_payment, index=test.ticket_id)

In [12]:
blight_model()

ticket_id
284932    0.030000
285362    0.000000
285361    0.000000
285338    0.060000
285346    0.000000
285345    0.110000
285347    0.040000
285342    0.360000
285530    0.015167
284989    0.020000
285344    0.000000
285343    0.000000
285340    0.160000
285341    0.286500
285349    0.030000
285348    0.040000
284991    0.000000
285532    0.130000
285406    0.010000
285001    0.000000
285006    0.010000
285405    0.010000
285337    0.010000
285496    0.000000
285497    0.050000
285378    0.000000
285589    0.000000
285585    0.020000
285501    0.104286
285581    0.220000
            ...   
376367    0.000000
376366    0.090000
376362    0.000000
376363    0.030000
376365    0.000000
376364    0.090000
376228    0.728667
376265    0.010000
376286    0.449000
376320    0.034667
376314    0.000000
376327    0.830000
376385    0.280000
376435    0.530000
376370    0.300000
376434    0.030000
376459    0.010000
376478    0.000000
376473    0.000000
376484    0.020000
376482    0.010000
37